# Generate DEM from USGS 3D Elevation Program (3DEP) lidar data for user-defined area of interest

<a name="Author"></a>
## Author
Bishal Dhungana

<a name="Install-and-run-on-local-file-system"></a>
### Install and run on local file system

Because of the file size, it is better to run the Jupyter Notebook on your local machine:

Make a new directory on your local file system where the post_lidar Jupyter Notebooks (and all products data, if desired) will be saved. In this example case, the directory will be called `post_lidar`.

    $ mkdir post_lidar

Change into the new directory and `git clone` the dev branch in the Github repository containing the Jupyter Notebooks and other relevant files to your local file system.

    $ cd post_lidar
    
    $ git clone 

Next, create a virtual environment with the required dependencies, using the `environment.yml` file contained in the cloned Github repo. Note: Executing the following command will automatically create the conda environment with name `venv` and all of the required dependencies installed. If you would prefer a different name, replace `venv` with another name in the following command:

    $ conda env create -n venv --file environment.yml

Activate the conda environment with all of the necessary dependencies installed:

    $ conda activate venv

Now, launch the chosen Jupyter Notebook. If unsure how to launch a Notebook, refer to this guide (https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/execute.html).

**You may now proceed to Library Imports**

<a name="Library-Imports"></a>
### Library Imports

After successfully completing the steps outlined above, we can now import the modules for use throughout the rest of the notebook.

In [ ]:
# Import Modules
import copy
import json
import math
import os
import re
import subprocess
from datetime import datetime

import numpy as np
import geopandas as gpd
import ipyleaflet
import ipywidgets as widgets
import pdal
import pyproj
import requests
import rasterio
import richdem as rd
from osgeo import gdal, ogr, osr
from shapely.geometry import shape, Point, Polygon
from shapely.ops import transform

import rioxarray as rxr
from rasterio.enums import Resampling
import matplotlib.pyplot as plt
import rasterio.warp

### Define AOI

In [ ]:
# Enter shapefile path, if applicable. Example: shapefile_path = '/path/to/shapefile.shp'.
# Otherwise leave as shapefile_path = ''
# Run this cell either way, or next cell will not run appropriately.
shapefile_path = "../shp/Himmel.shp" # /path/to/your/shapefile/shapefile.shp

<a name="Define-Functions"></a>
### Define Functions

Several functions are provided in the cell below. These functions are necessary for successful execution of remainder of the notebook. Broadly, these functions provide the utility for the user to draw and area of interest (AOI) on an interactive map and construct the PDAL pipeline for getting the point cloud data from the Amazon Web Services EPT bucket, performing processing steps, producing DEM, and saving the results.

In [ ]:
def proj_to_3857(poly, orig_crs):
    wgs84 = pyproj.CRS("EPSG:4326")
    web_mercator = pyproj.CRS("EPSG:3857")
    project_gcs = pyproj.Transformer.from_crs(orig_crs, wgs84, always_xy=True).transform
    project_wm = pyproj.Transformer.from_crs(orig_crs, web_mercator, always_xy=True).transform
    user_poly_proj4326 = transform(project_gcs, poly)
    user_poly_proj3857 = transform(project_wm, poly)
    return(user_poly_proj4326, user_poly_proj3857)

def gcs_to_proj(poly):
    wgs84 = pyproj.CRS("EPSG:4326")
    web_mercator = pyproj.CRS("EPSG:3857")
    project = pyproj.Transformer.from_crs(wgs84, web_mercator, always_xy=True).transform
    user_poly_proj3857 = transform(project, poly)
    return(user_poly_proj3857)

def import_shapefile_to_shapely(path):
    shapefile_path = path
    gdf = gpd.read_file(shapefile_path)
    orig_crs = gdf.crs                   # this is the original CRS of the imported shapefile
    user_shp = gdf.loc[0, 'geometry']
    user_shp_epsg4326, user_shp_epsg3857 = proj_to_3857(user_shp, orig_crs)
    user_AOI = [[user_shp_epsg4326, user_shp_epsg3857]]
    return user_AOI
    
def handle_draw(target, action, geo_json):      
    geom = dict(geo_json['geometry'])
    user_poly = shape(geom)
    user_poly_proj3857 = gcs_to_proj(user_poly)
    print('AOI is valid and has boundaries of ', user_poly_proj3857.bounds, 'Please proceed to the next cell.')
    user_AOI.append((user_poly, user_poly_proj3857))  #for various reasons, we need user AOI in GCS and EPSG 3857
    
def downsample_dem(dem, target_resolution=10):
    # Get the current resolution
    current_resolution_x = abs(dem.rio.resolution()[0])
    current_resolution_y = abs(dem.rio.resolution()[1])
    
    # Calculate the scale factors
    scale_factor_x = current_resolution_x / target_resolution
    scale_factor_y = current_resolution_y / target_resolution
    
    # Calculate the new width and height based on the scale factors
    new_width = int(dem.rio.width * scale_factor_x)
    new_height = int(dem.rio.height * scale_factor_y)
    
    # Downsample DTM/DSM
    down_sampled = dem.rio.reproject(
        dem.rio.crs,
        shape=(new_height, new_width),
        resampling=Resampling.bilinear
    )
    
    return down_sampled
    
def build_pdal_pipeline(extent_epsg3857, usgs_3dep_dataset_names, pc_resolution, filterNoise = False,
                        reclassify = False, savePointCloud = True, outCRS = 3857, pc_outName = 'filter_test',
                        pc_outType = 'laz'):

    #this is the basic pipeline which only accesses the 3DEP data
    readers = []
    for name in usgs_3dep_dataset_names:
        url = "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/{}/ept.json".format(name)
        reader = {
            "type": "readers.ept",
            "filename": str(url),
            "polygon": str(extent_epsg3857),
            "requests": 3,
            "resolution": pc_resolution
        }
        readers.append(reader)
        
    pointcloud_pipeline = {
            "pipeline":
                readers
    }
    
    if filterNoise == True:
        
        # Filter stage for class 7
        filter_stage_class7 = {
            "type": "filters.range",
            "limits": "Classification![7:7]"
        }

        # Filter stage for class 18
        filter_stage_class18 = {
            "type": "filters.range",
            "limits": "Classification![18:18]"
        }

        # Append both filter stages to the pipeline separately
        pointcloud_pipeline['pipeline'].append(filter_stage_class7)
        pointcloud_pipeline['pipeline'].append(filter_stage_class18)
    
    if reclassify == True:
        
        remove_classes_stage = {
            "type":"filters.assign",
            "value":"Classification = 0"
        }
        
        classify_ground_stage = {
            "type":"filters.smrf"
        }
        
        reclass_stage = {
            "type":"filters.range",
            "limits":"Classification[2:2]"
        }

        pointcloud_pipeline['pipeline'].append(remove_classes_stage)
        pointcloud_pipeline['pipeline'].append(classify_ground_stage)
        pointcloud_pipeline['pipeline'].append(reclass_stage)
        
    reprojection_stage = {
        "type":"filters.reprojection",
        "out_srs":"EPSG:{}".format(outCRS)
    }
    
    pointcloud_pipeline['pipeline'].append(reprojection_stage)
    
    if savePointCloud == True:
        
        if pc_outType == 'las':
            savePC_stage = {
                "type": "writers.las",
                "filename": str(pc_outName)+'.'+ str(pc_outType),
            }
        elif pc_outType == 'laz':    
            savePC_stage = {
                "type": "writers.las",
                "compression": "laszip",
                "filename": str(pc_outName)+'.'+ str(pc_outType),
            }
        else:
            raise Exception("pc_outType must be 'las' or 'laz'.")

        pointcloud_pipeline['pipeline'].append(savePC_stage)
        
    return pointcloud_pipeline

def make_DEM_pipeline(extent_epsg3857, usgs_3dep_dataset_name, pc_resolution, dem_resolution,
                      filterNoise=True, reclassify=False, savePointCloud=False, outCRS=3857,
                      pc_outName='filter_test', pc_outType='laz', demType='dtm', gridMethod='idw', 
                      dem_outName='dem_test', dem_outExt='tif', driver="GTiff"):
    
    dem_pipeline = build_pdal_pipeline(extent_epsg3857, usgs_3dep_dataset_name, pc_resolution,
                                       filterNoise, reclassify, savePointCloud, outCRS, pc_outName, pc_outType)
    
    if demType == 'dsm':
        dem_stage = {
            "type": "writers.gdal",
            "filename": str(dem_outName) + '.' + str(dem_outExt),
            "gdaldriver": driver,
            "nodata": -9999,
            "output_type": gridMethod,
            "resolution": float(dem_resolution),
            "gdalopts": "COMPRESS=LZW,TILED=YES,blockxsize=256,blockysize=256,COPY_SRC_OVERVIEWS=YES"
        }
    
    elif demType == 'dtm':
        groundfilter_stage = {
            "type": "filters.range",
            "limits": "Classification[2:2]"
        }

        dem_pipeline['pipeline'].append(groundfilter_stage)

        dem_stage = {
            "type": "writers.gdal",
            "filename": str(dem_outName) + '.' + str(dem_outExt),
            "gdaldriver": driver,
            "nodata": -9999,
            "output_type": gridMethod,
            "resolution": float(dem_resolution),
            "gdalopts": "COMPRESS=LZW,TILED=YES,blockxsize=256,blockysize=256,COPY_SRC_OVERVIEWS=YES"
        }
    
    else:
        raise Exception("demType must be 'dsm' or 'dtm'.")
        
    dem_pipeline['pipeline'].append(dem_stage)
    
    return dem_pipeline

<a name="Data-Access-and-Processing"></a>
## Data Access and Processing
Now that we have the required modules imported and functions defined, we can proceed with defining our area of interest (AOI), accessing/processing the 3DEP data from the Amazon Web Services EPT bucket. 

In [ ]:
# Get GeoJSON file for 3DEP outlines from URL 

print("Requesting, loading, and projecting 3DEP dataset polygons...")

#request the boundaries from the Github repo and save locally.
url = 'https://raw.githubusercontent.com/hobuinc/usgs-lidar/master/boundaries/resources.geojson'
r = requests.get(url, verify=False)
with open('resources.geojson', 'w') as f:
    f.write(r.content.decode("utf-8"))

with open('resources.geojson', 'r') as f:
    geojsons_3DEP = json.load(f)
    
#make pandas dataframe and create pandas.Series objects for the names, urls, and number of points for each boundary.
with open('resources.geojson', 'r') as f:
    df = gpd.read_file(f)
    names = df['name']
    urls = df['url']
    num_points = df['count']

#project the boundaries to EPSG 3857 (necessary for API call to AWS for 3DEP data)
projected_geoms = []
for geometry in df['geometry']:
        projected_geoms.append(gcs_to_proj(geometry))

geometries_GCS = df['geometry']
geometries_EPSG3857 = gpd.GeoSeries(projected_geoms)

print('Done. 3DEP polygons downloaded and projected to Web Mercator (EPSG:3857)')

In [ ]:
# If you don't have a shapefile, select an area of interest using the tools on the left side of the map.

m = ipyleaflet.Map(
    basemap=ipyleaflet.basemaps.Esri.WorldTopoMap,
    center=(37, -100),
    zoom=3.5,
    crs=ipyleaflet.projections.EPSG3857
    )

geo_json_3DEP = ipyleaflet.GeoJSON(data=geojsons_3DEP, style = {'color': 'green', 'opacity':1, 
                                       'weight':1, 'fillOpacity':0.1})

m.add_layer(geo_json_3DEP)  #add 3DEP polygons GeoJSON

dc = ipyleaflet.DrawControl(
    polygon={"shapeOptions": {"color": "blue"}},
    rectangle={"shapeOptions": {"color": "blue"}},
    circlemarker={},
    polyline={}
)

if os.path.exists(shapefile_path):
    user_AOI = import_shapefile_to_shapely(shapefile_path)
    print('shapefile loaded. proceed to next cell')
    
else:
    print('Select an Area of Interest using the tools on the left side of the map.')
    user_AOI = []
    dc.on_draw(handle_draw)
    m.add_control(dc)
    display(m)

In [ ]:
# We need to find the most recent 3DEP data that covers the AOI. The aim is to get the most recent as well as the data from a single vendor or flight.
# If you want to get all the data regardless of vendor or year, comment out the next cell.

# Function to extract year from URL
def extract_year_from_url(url):
    match = re.search(r'(\d{4})', url)
    if match:
        return int(match.group(1))
    return None

AOI_GCS = user_AOI[-1][0]
AOI_StatePlane = user_AOI[-1][1]

intersecting_polys = []

# Collect intersecting polygons with their years
polygons_with_years = []
for i, geom in enumerate(geometries_StatePlane):
    if geom.intersects(AOI_StatePlane):
        current_year = extract_year_from_url(urls[i])
        if current_year:
            polygons_with_years.append((current_year, names[i], geometries_GCS[i], geometries_StatePlane[i], urls[i], num_points[i]))

# Sort polygons by year in descending order
polygons_with_years.sort(reverse=True, key=lambda x: x[0])

# Check if the AOI is completely covered by the latest polygons
covered_area = Polygon()
for poly in polygons_with_years:
    if covered_area.covers(AOI_StatePlane):
        break
    covered_area = covered_area.union(poly[3])
    intersecting_polys.append(poly)

# Print the intersecting polygons
for poly in intersecting_polys:
    poly_json = json.dumps({
        "name": poly[1],
        "geometry_GCS": poly[2].__geo_interface__,
        "geometry_StatePlane": poly[3].__geo_interface__,
        "url": poly[4],
        "num_points": int(poly[5])
    }, indent=4)
    print(poly_json)

# Proceed to the next step if AOI falls completely within the polygon
if covered_area.covers(AOI_StatePlane):
    print("AOI is completely covered by the polygons. Proceeding to the next step.")
else:
    print("AOI is not completely covered by the polygons. Additional polygons may be needed.")

In [ ]:
# # If you want to get all the data regardless of vendor or year, comment out this cell. 

# AOI_GCS = user_AOI[-1][0]
# AOI_EPSG3857 = user_AOI[-1][1]

# intersecting_polys = []

# for i,geom in enumerate(geometries_EPSG3857):
#     if geom.intersects(AOI_EPSG3857):
#         intersecting_polys.append((names[i], geometries_GCS[i], geometries_EPSG3857[i], urls[i], num_points[i]))
        
# print(intersecting_polys)

In [ ]:
# Find AOI center for plotting purposes
centroid = list(AOI_GCS.centroid.coords)[0]

# Make ipyleaflet map
m = ipyleaflet.Map(
    basemap=ipyleaflet.basemaps.Esri.WorldTopoMap,
    center=(centroid[1], centroid[0]),
    zoom=12,
)

# Add intersecting 3DEP polygon(s) to the map
wlayer_3DEP_list = []
usgs_3dep_datasets = []
number_pts_est = []
for i, poly in enumerate(intersecting_polys):
    wlayer_3DEP = ipyleaflet.WKTLayer(
        wkt_string=poly[2].wkt,  # Use the correct geometry object
        style={"color": "green"}
    )
    
    m.add_layer(wlayer_3DEP)
    wlayer_3DEP_list.append(wlayer_3DEP)
    usgs_3dep_datasets.append(poly[1])
    
    # Estimate total points using ratio of area and point count
    number_pts_est.append((int((AOI_StatePlane.area / poly[3].area) * (poly[5]))))

# Make ipyleaflet layers from the AOI and add to map
wlayer_user = ipyleaflet.WKTLayer(
    wkt_string=AOI_GCS.boundary.wkt,
    style={"color": "blue"}
)

AOI_StatePlane_wkt = AOI_StatePlane.wkt
m.add_layer(wlayer_user)

# Sum the estimates of the number of points from each 3DEP dataset within the AOI
num_pts_est = sum(number_pts_est)

# Plot map and specify desired point cloud resolution using a widget
user_resolution = widgets.RadioButtons(
    options=[
        (f'Full - All ~{int(math.ceil(num_pts_est / 1e6) * 1e6):,} points', 1.0),
        (f'High - 2m resolution', 2.0),
        (f'Mid  - 5m resolution', 5.0),
        (f'Low  - 10m resolution', 10.0)
    ],
    layout={'width': 'max-content'},
    disabled=False,
)

display(m)
print(f'Your AOI at full resolution will include approximately {int(math.ceil(num_pts_est / 1e6) * 1e6):,} points. Select desired point cloud resolution.')
widgets.VBox([user_resolution])

In [ ]:
# Do not modify AOI_EPSG3857_wkt, usgs_3dep_datasets, or pointcloud_resolution
# Modify the optional arguments to fit user need.
# Change outCRS to EPSG code of desired coordinate reference system (Default is EPSG:3857 - Web Mercator Projection)
# Change pc_outname to descriptive name and pc_outType to 'las' or 'laz'.
os.chdir("../pc")
pointcloud_resolution = user_resolution.value
pc_pipeline = build_pdal_pipeline(AOI_EPSG3857_wkt, usgs_3dep_datasets, pointcloud_resolution, filterNoise = True,
                                  reclassify = False, savePointCloud = True, outCRS = 3857, 
                                  pc_outName = 'pointcloud_test', pc_outType = 'laz')

The PDAL pipeline is now constructed. Running the the PDAL Python bindings function ```pdal.Pipeline()``` creates the pdal.Pipeline object from a json-ized version of the pointcloud pipeline we created.

In [ ]:
pc_pipeline = pdal.Pipeline(json.dumps(pc_pipeline))

In [ ]:
%%time
pc_pipeline.execute_streaming(chunk_size=1000000) # use this if reclassify == False 
#pc_pipeline.execute() # use this if reclassify == True 

If the user only desires point cloud data, they may stop here. Following is an overview on how dtm may be created.

<a name="Make-Digital-Terrain-Model-(DTM)"></a>
### Make Digital Terrain Model (DTM)
The following cells will produce a Digital Terrain Model (DTM), also called a 'bare earth model' using lidar returns classified as 'ground' (USGS Class 2). Do not modify the `AOI_EPSG3857_wkt`, `usgs_3dep_datasets`, or `pointcloud_resolution` arguments. Specify the desired dtm resolution (in meters), the appropriate point cloud processing steps, and the file names/extensions.

In [ ]:
# Do not modify AOI_EPSG3857_wkt, usgs_3dep_datasets, or pointcloud_resolution
# Modify the optional arguments to fit user need.
# Change outCRS to EPSG code of desired coordinate reference system (Default is EPSG:3857 - Web Mercator Projection)
# Change dem_outname to descriptive name and change dem_outExt and driver to desired file type.
os.chdir("../tif")
pointcloud_resolution = user_resolution.value
dtm_resolution = 0.3048  # 1 foot resolution
dtm_pipeline = make_DEM_pipeline(AOI_EPSG3857_wkt, usgs_3dep_datasets, pointcloud_resolution, dtm_resolution,
                                 filterNoise = True, reclassify = False, savePointCloud = False, outCRS = 3857,
                                 pc_outName = 'pointcloud_test', pc_outType = 'laz', demType = 'dtm', 
                                 gridMethod='idw', dem_outName = 'test_dtm', dem_outExt = 'tif', driver = "GTiff")

The PDAL pipeline is now constructed for making the DTM. Running the the PDAL Python bindings function ```pdal.Pipeline()``` creates the pdal.Pipeline object from a json-ized version of the pointcloud pipeline we created.

In [ ]:
dtm_pipeline = pdal.Pipeline(json.dumps(dtm_pipeline))

In [ ]:
%%time
dtm_pipeline.execute_streaming(chunk_size=1000000) # use this if reclassify == False 
#dtm_pipeline.execute() # use this if reclassify == True

### Credits
This notebook is based on the one made by Cole Speed, Matthew Beckley, Christopher Crosby (UNAVCO, Inc.), and Viswanath Nandigam (San Diego Supercomputer Center). Some of the contents and tools provided here are based on their original work and contributions.